# Coletar dados de API

Este notebook realiza a extração dos dados via API pública e salva os dados na camada Landing (LND), no formato JSON.

## Importações e configurações

In [5]:
import requests
import json
import os
from datetime import date
from typing import Any

In [ ]:
def salvar_json(response: requests.Response, file_name: str) -> None:
    """
    Salva os dados de uma resposta JSON da API em um arquivo na estrutura de diretórios da camada LND.

    Args:
        response (requests.Response): Objeto de resposta da requisição HTTP.
        file_name (str): Nome da subpasta e prefixo do arquivo a ser salvo na pasta LND.

    Returns:
        None
    """
    if response.status_code == 200:
        data: Any = response.json()

        path = f"../LND/{file_name}"
        os.makedirs(path, exist_ok=True)

        filename = f"{path}/{file_name}_{date.today()}.json"

        with open(filename, "w", encoding="utf-8") as file:
            json.dump(data, file, indent=4, ensure_ascii=False)
        
        print(f"Arquivo salvo em: {filename}")
    else:
        print(f"Erro ao acessar a API: {response.status_code}")

## Dados IBGE

In [ ]:
url: str = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
response: requests.Response = requests.get(url)

In [8]:
salvar_json(response, "municipios")

Arquivo salvo em: ../LND/municipios/municipios_2025-07-17.json


## Outros dados complementares

Os dados complementares necessários para este projeto — como Natureza Jurídica, CNAE, Simples Nacional, Municípios da Receita, entre outros — não estão disponíveis via API pública. No entanto, podem ser obtidos diretamente no portal da Receita Federal através de arquivos CSV.

📂 Acesse: [Base CNPJ — Receita Federal (jun/2025)](https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/)

Esses arquivos devem ser considerados parte da **camada Landing (LND)** e posteriormente processados nos notebooks seguintes.